In [ ]:
import mezze
import numpy as np

The first use case considered is to execute a simulation of 1 second simulation time with 1000 time steps, i.e. $\Delta T$=0.001 sec.  100 Monte Carlo runs are performed in a parallel fashion.  For purposes of this demonstration, zero-valued controls are used, creating a (very) noisy identity channel.

In [ ]:
config = mezze.simulation.SimulationConfig(time_length=1.,num_steps=1000)
config.parallel = True
config.num_runs = 100
pmd = mezze.implementations.XYZ_noise_XY_control_Qubit()
ham = mezze.hamiltonian.HamiltonianFunction(pmd)
ctrls = np.zeros((1000,3))
pham = mezze.hamiltonian.PrecomputedHamiltonian(pmd, ctrls, config, ham.get_function())
sim = mezze.simulation.Simulation(config,pmd,pham)

The following block actually executes the simulation and returns a report containing the output.

In [ ]:
report = sim.run()

The key element of this report is a `QuantumChannel` object that is the Monte Carlo estimate of the specified PMD and input control.

In [ ]:
report.channel.liouvillian()

In [ ]:
report.channel.chi()

The next example is similar to the first, except that we have configured the simulation to store *both* each monte carlo realization, as well as to include incremental data taken every 500 time steps.  These can be used independently.  For illustration purposes, only 3 Monte Carlo runs will be performed.

In [ ]:
config = mezze.simulation.SimulationConfig(time_length=1.,num_steps=1000)
config.parallel = True
config.num_runs = 3

# Store all the samples being averaged over
config.realization_sampling = True

# Also keep intermediate realizations every 500 time steps
config.time_sampling = True
config.sampling_interval = 500

pmd = mezze.implementations.XYZ_noise_XY_control_Qubit()
ham = mezze.hamiltonian.HamiltonianFunction(pmd)
ctrls = np.zeros((1000,2))
pham = mezze.hamiltonian.PrecomputedHamiltonian(pmd, ctrls, config, ham.get_function())
sim = mezze.simulation.Simulation(config,pmd,pham)

In [ ]:
report = sim.run()

`report.channel` still contains the Monte Carlo estimate of the gate at the specified end time of the simulation.

In [ ]:
report.channel

The individual samples that are averaged in to `report.channel` are in `report.liouvillian_samples`. Note there are 3, as specified in `config.num_runs`

In [ ]:
report.liouvillian_samples

The sub-sampled trajectories are stored in `report.time_samples`.  Since the sampling interval was set to 500, each of the 3 Monte Carlo trajectories is sampled 1000/500=2 times.  If `config.realization_sampling` were set to false, a list containing the average channel each increment would be reported.

In [ ]:
report.time_samples

As a final example, this time we set a convergence tolerance (far too high for real usage), rather than a fixed number of runs.  While this mode is compatible with `realization_sampling` the number of runs required for a small convergence criterion will likely result in too many samples to comfortably store.

In [ ]:
config = mezze.simulation.SimulationConfig(time_length=1.,num_steps=1000)
config.parallel = True

#Do blocks of runs until convergence tolerance reached
config.num_runs = 4# Set to number of cpus you want to use
config.run_to_convergence = True
config.convergence_tolerance = 1e-3
config.num_runs_below_tolerance = 1

pmd = mezze.implementations.XYZ_noise_XY_control_Qubit()
ham = mezze.hamiltonian.HamiltonianFunction(pmd)
ctrls = np.zeros((1000,2))
pham = mezze.hamiltonian.PrecomputedHamiltonian(pmd, ctrls, config, ham.get_function())
sim = mezze.simulation.Simulation(config,pmd,pham)

In [ ]:
report = sim.run()

`report.num_runs` keeps track of the total number of Monte Carlo runs performed, which is really only informative when a convergence tolerance is used.

In [ ]:
print(report.num_runs)